# Multiple-objective portfolio optimization

# INTRODUCTION

Your task is to solve a multiple-objective portfolio optimization problem.
-  Use the basic Markowitz's model from 1952 (see Lecture 1)
-  Solve = construct Pareto front approximations.
-  The dataset is the same as for the portfolio game part 1 (bundle1.zip).
-  The dataset consists of the historical prices of 20 assets.
-  The bundle contains 20 files (*.txt) linked to different assets.
-  The name of the file suggests the asset's name.
-  The structure of every file is as follows:
1.  The first line contains the name of the asset.
2. The second line provides the number of data points N.
3. The following N lines are data points with the structure: time, price.
-  The historical timeline for all assets is time $\in$ [0,100].
-  Future predictions should be calculated for time = 200.

Goal: 
-  Load data, make predictions, and build the model. 
-  Illustrate your predictions (can be done in the jupyter notebook)
-  Then, implement the WSM and ECM methods (see the tutorial on quadratic programming provided below). 
-  Run your implementations for different calculation limits (e.g., the number of weight vectors for WSM). Compare the methods' efficiency in finding unique Pareto optimal solutions. Finally, illustrate generated Pareto fronts.

# Short tutorial on the cvxopt library for quadratic programming

In [60]:
import numpy as np
from cvxopt import matrix, solvers

# QP Optimization Problem

### General model:

$max$ $\boldsymbol{cx} - \dfrac{1}{2}\boldsymbol{x}^T\boldsymbol{Qx}$ <br>
$s.t.$ <br>
$\boldsymbol{Gx} \leq \boldsymbol{h}$ <br>
$\boldsymbol{x} \geq \boldsymbol{0}$

### But the library uses the following form:

$min$ $\boldsymbol{cx} + \dfrac{1}{2}\boldsymbol{x}^T\boldsymbol{Qx}$ <br>
$s.t.$ <br>
$\boldsymbol{Gx} \leq \boldsymbol{h}$ <br>
$\boldsymbol{Ax} = \boldsymbol{b}$ <br>

### Exmple

$min$ $2x^2_1+x_2^2+x_1x_2+x_1+x_2$ <br>
$s.t.$ <br>
$x_1 \geq 0$<br>
$x_2 \geq 0$<br>
$x_1 + x_2 = 1$<br>

### Hence:

In [61]:
Q = matrix([ [4.0, 1.0], [1.0, 2.0] ]) ## [4, 1] is 1st column, not row!

In [62]:
c = matrix([1.0, 1.0]) ### (1, 2) = dimensions (1 row and 2 columns)

In [63]:
A = matrix([1.0, 1.0], (1,2)) ### (1, 2) = dimensions (1 row and 2 columns)

In [64]:
b = matrix(1.0) 

In [65]:
G = matrix([[-1.0,0.0],[0.0,-1.0]]) ### multiplied both sides by -1

In [66]:
h = matrix([0.0,0.0]) ### multiplied both sides by -1

In [67]:
solQP=solvers.qp(Q, c, G, h, A, b)

In [68]:
print(solQP.keys())

dict_keys(['x', 'y', 's', 'z', 'status', 'gap', 'relative gap', 'primal objective', 'dual objective', 'primal infeasibility', 'dual infeasibility', 'primal slack', 'dual slack', 'iterations'])


In [69]:
print(solQP['x'])
print(solQP['primal objective'])

[ 2.50e-01]
[ 7.50e-01]

1.8750000000000182


# We can also solve LP problems:

$min$ $\boldsymbol{c}\boldsymbol{x}$ <br>
$s.t.$ <br>
$\boldsymbol{Gx} \leq \boldsymbol{h}$ <br>
$\boldsymbol{Ax} = \boldsymbol{b}$ (optional)

### Exmple

$min$ $2x_1+x_2$ <br>
$s.t.$ <br>
$-x_1 +x_2 \leq 1$ <br>
$x_1 + x_2 \geq 2$ <br>
$x_2 \geq 0$<br>
$x_1 - 2x_2 \leq 4$

In [70]:
G = matrix([ [-1.0, -1.0, 0.0, 1.0], [1.0, -1.0, -1.0, -2.0] ])
h = matrix([ 1.0, -2.0, 0.0, 4.0 ])
c = matrix([ 2.0, 1.0 ])
solLP = solvers.lp(c,G,h)  
###!!!! OPTIONALLY A and b can be provided (equality constraints) as in solQP=solvers.qp(Q, c, G, h, A, b)

In [71]:
print(solLP.keys())

dict_keys(['x', 'y', 's', 'z', 'status', 'gap', 'relative gap', 'primal objective', 'dual objective', 'primal infeasibility', 'dual infeasibility', 'primal slack', 'dual slack', 'residual as primal infeasibility certificate', 'residual as dual infeasibility certificate', 'iterations'])


In [72]:
print(solLP['x'])
print(solLP['primal objective'])

[ 5.00e-01]
[ 1.50e+00]

2.499999989554308


# Portfolio optimization

Import the necessary libraries (numpy, pandas, matplotlib, cvxopt).

In [73]:
################################################################################
# CELL 1: Imports
################################################################################
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from cvxopt import matrix, solvers

# For reproducible randomness (if needed)
np.random.seed(42)

Load the actual data from your 20 (or more) *Part1.txt files. Each file’s first line is the asset name, second line is N, and then N lines with “time price”.

In [74]:
################################################################################
# CELL 2: Data Loading
#   - We assume you have ~20 text files in a "data" folder, each named *Part1.txt
#   - Structure of each file:
#       1) Asset name (string)
#       2) Number of data points N
#       3) N lines of "time price"
################################################################################

data_folder = "data"

# Gather all Part1.txt files in the data folder
txt_files = [f for f in os.listdir(data_folder) if f.endswith("Part1.txt")]

asset_names = []
asset_times = []
asset_prices = []

for fname in txt_files:
    path = os.path.join(data_folder, fname)
    with open(path, "r") as f:
        # 1) asset name
        asset_name = f.readline().strip()

        # 2) number of data points
        N_line = f.readline().strip()
        N = int(N_line)

        # 3) read time, price lines
        times = []
        prices = []
        for _ in range(N):
            line = f.readline().strip()
            t_str, p_str = line.split()
            times.append(float(t_str))
            prices.append(float(p_str))

        asset_names.append(asset_name)
        asset_times.append(times)
        asset_prices.append(prices)

print(f"Found {len(asset_names)} assets.")
print("First few asset names:", asset_names[:5])

Found 20 assets.
First few asset names: ['SafeAndCare', 'Moneymakers', 'Fuel4', 'CPU-XYZ', 'MarsProject']


Perform a simple linear regression (degree=1) for each asset using times in [0,100]. Extrapolate to time=200 to get a predicted price. Convert that predicted price growth into a predicted return \mu[i].

In [75]:
################################################################################
# CELL 3: Forecasting / Predictions
#   - We'll do a simple linear regression for each asset using data up to time=100.
#   - We'll extrapolate to predict the price at time=200.
#   - Then, interpret that predicted growth from t=100 to t=200 as an *expected return*.
#
#   A more sophisticated approach might do:
#       * polynomial fits
#       * log-transformed fits
#       * ARIMA or other time series modeling
################################################################################

predicted_prices_200 = []
current_prices_100 = []

for i in range(len(asset_names)):
    times = np.array(asset_times[i])
    prices = np.array(asset_prices[i])

    # Fit a line: price(t) ~ a * t + b
    # We'll filter times up to t=100. (They should all be up to 100 anyway.)
    # But let's just ensure we only use [0..100].
    # If data indeed stops at exactly 100, we use them all.
    mask = (times >= 0) & (times <= 100)
    t_used = times[mask]
    p_used = prices[mask]

    # Perform linear regression with np.polyfit (degree=1)
    coeffs = np.polyfit(t_used, p_used, deg=1)
    # coeffs = [a, b] => price(t) = a*t + b

    # Evaluate the fitted line at t=100 and t=200
    price_at_100 = np.polyval(coeffs, 100)
    price_at_200 = np.polyval(coeffs, 200)

    predicted_prices_200.append(price_at_200)
    current_prices_100.append(price_at_100)

# Convert to arrays
predicted_prices_200 = np.array(predicted_prices_200)
current_prices_100 = np.array(current_prices_100)

# Example: The "forecasted return" from time=100 to time=200
# We'll define a simple measure: r_i = (price200 - price100)/price100
predicted_returns = (predicted_prices_200 - current_prices_100) / current_prices_100

# Let's illustrate the first few predictions:
for i in range(min(3, len(asset_names))):
    print(f"Asset: {asset_names[i]}")
    print(f"   Fitted price at t=100: {current_prices_100[i]:.3f}")
    print(f"   Predicted price at t=200: {predicted_prices_200[i]:.3f}")
    print(f"   => Predicted return = {predicted_returns[i]:.2%}")

Asset: SafeAndCare
   Fitted price at t=100: 6.455
   Predicted price at t=200: 5.575
   => Predicted return = -13.64%
Asset: Moneymakers
   Fitted price at t=100: 4.580
   Predicted price at t=200: 6.541
   => Predicted return = 42.83%
Asset: Fuel4
   Fitted price at t=100: 5.643
   Predicted price at t=200: 3.480
   => Predicted return = -38.34%


Illustrate your predictions by plotting the historical data and the linear fit for a few assets, plus the forecasted price at t=200.

In [ ]:
################################################################################
# CELL 4: Illustrate Predictions
#   - Let's plot the linear fit vs. actual data for a couple of assets, just as proof-of-concept.
################################################################################

num_to_plot = min(3, len(asset_names))  # plot a few assets
for i in range(num_to_plot):
    times = np.array(asset_times[i])
    prices = np.array(asset_prices[i])

    # Fit again
    coeffs = np.polyfit(times, prices, deg=1)
    fit_line = np.polyval(coeffs, times)

    plt.figure()
    plt.scatter(times, prices, label="Historical prices")
    plt.plot(times, fit_line, label="Linear fit")

    # We'll also show the predicted price at t=200
    price_200 = np.polyval(coeffs, 200)
    plt.scatter([200], [price_200], marker='x', label="Forecast @ t=200")

    plt.title(f"Price vs Time for {asset_names[i]}")
    plt.xlabel("Time")
    plt.ylabel("Price")
    plt.legend()
    plt.show()

Build the Markowitz model inputs.
	•	\mu[i] comes from the predicted returns (time=100 to time=200).
	•	\Sigma is estimated from the historical returns from the adjacency in your original price data (0..100).

In [77]:
################################################################################
# CELL 5: Build Markowitz Model Inputs
#   - We have a predicted return for each asset from t=100 -> t=200: predicted_returns
#       => We can treat this as our "mu[i]" (expected return).
#   - Next, we need a covariance matrix Sigma from historical returns (0..100).
#   - We'll do standard sample covariance from discrete returns from t_{k} to t_{k+1}.
################################################################################

# Let's compute historical returns from the raw price data for times in [0..100].
# We'll treat each adjacent time step as a "return."
# We assume each asset has the same # of points from t=0..100 for simplicity.

all_historical_returns = []  # shape: (T, n_assets), T ~ 100 for example

n_assets = len(asset_names)
max_periods = None

for i in range(n_assets):
    p = np.array(asset_prices[i])
    # We'll do the discrete return for consecutive data points:
    # r_k = (p_{k+1}-p_k)/p_k
    # so if the asset has M points, we get M-1 returns
    returns_i = []
    for k in range(len(p) - 1):
        if p[k] != 0:
            ret_k = (p[k+1] - p[k]) / p[k]
        else:
            ret_k = 0.0
        returns_i.append(ret_k)
    if max_periods is None:
        max_periods = len(returns_i)
    else:
        max_periods = min(max_periods, len(returns_i))
    all_historical_returns.append(returns_i)

# We should align them to the same number of time steps, if needed
# to keep them consistent. We'll truncate to the smallest length among all assets.
for i in range(n_assets):
    all_historical_returns[i] = all_historical_returns[i][:max_periods]

# Convert to array shape (max_periods, n_assets)
all_historical_returns = np.array(all_historical_returns).T  # shape (max_periods, n_assets)

print("all_historical_returns shape:", all_historical_returns.shape)

# Now compute sample mean and covariance of these historical returns.
# But we won't use the sample mean. We have a forecast from time=100->200, predicted_returns.
# We'll only use the covariance. So let's do:
Sigma = np.cov(all_historical_returns, rowvar=False)  # shape (n_assets, n_assets)

mu = predicted_returns  # from forecasting

all_historical_returns shape: (100, 20)


Implement the two multi-objective methods:
	•	Weighted Sum Method (WSM): For each \alpha\in[0,1], solve \min \alpha (x^T\Sigma x) - (1-\alpha) (\mu^T x).
	•	Epsilon-Constraint Method (ECM): For each target return r, solve \min x^T\Sigma x subject to \mu^T x \ge r.

In [78]:
################################################################################
# CELL 6: Implement WSM and ECM
#   Weighted Sum Method (WSM):
#     min alpha * x^T Sigma x  - (1 - alpha)*mu^T x
#     s.t. sum(x_i)=1, x_i >= 0
#   Epsilon Constraint Method (ECM):
#     min x^T Sigma x
#     s.t. mu^T x >= r
#          sum(x_i)=1, x_i >= 0
#
# We'll define 2 functions that solve these for different alpha or r values.
################################################################################

def solve_qp_cvxopt(Q, c, A=None, b=None, G=None, h=None, show_progress=False):
    """
    Minimizes (1/2)* x^T Q x + c^T x
    s.t. Gx <= h
         A x = b
    """
    solvers.options['show_progress'] = show_progress

    Q_cvx = matrix(Q, tc='d')
    c_cvx = matrix(c, tc='d')
    A_cvx = matrix(A, tc='d') if A is not None else None
    b_cvx = matrix(b, tc='d') if b is not None else None
    G_cvx = matrix(G, tc='d') if G is not None else None
    h_cvx = matrix(h, tc='d') if h is not None else None

    solution = solvers.qp(Q_cvx, c_cvx, G_cvx, h_cvx, A_cvx, b_cvx)
    x_opt = np.array(solution['x']).flatten()
    return x_opt, solution['primal objective']

def wsm_portfolios(mu, Sigma, num_points=11):
    """
    Weighted Sum Method
    We'll vary alpha in [0,1] with 'num_points' steps
    """
    n = len(mu)

    # constraints: sum(x) = 1, x >= 0
    A = np.ones((1, n))
    b = np.array([1.0])
    G = -1.0 * np.eye(n)
    h = np.zeros(n)

    alpha_list = np.linspace(0, 1, num_points)
    results_risk = []
    results_return = []
    results_weights = []

    for alpha in alpha_list:
        # objective = alpha * x^T Sigma x - (1-alpha)* mu^T x
        # in cvxopt form: 0.5 * x^T Q x + c^T x
        # => Q = 2*alpha*Sigma  (so that 0.5 * Q = alpha*Sigma)
        # => c = - (1-alpha)* mu
        Q_eff = 2.0 * alpha * Sigma
        c_eff = -(1.0 - alpha) * mu

        x_opt, _ = solve_qp_cvxopt(Q_eff, c_eff, A=A, b=b, G=G, h=h)
        # compute risk, return
        risk_val = x_opt @ Sigma @ x_opt
        ret_val = x_opt @ mu

        results_risk.append(risk_val)
        results_return.append(ret_val)
        results_weights.append(x_opt)

    return np.array(results_risk), np.array(results_return), np.array(results_weights)

def ecm_portfolios(mu, Sigma, num_points=11):
    """
    Epsilon Constraint Method
    We pick r in [min(mu), max(mu)] with 'num_points' steps
    Then solve:
      min x^T Sigma x
      s.t. mu^T x >= r, sum(x)=1, x>=0
    """
    n = len(mu)
    A = np.ones((1, n))
    b = np.array([1.0])
    base_G = -1.0 * np.eye(n)
    base_h = np.zeros(n)

    r_min, r_max = mu.min(), mu.max()
    r_values = np.linspace(r_min, r_max, num_points)

    results_risk = []
    results_return = []
    results_weights = []

    for r in r_values:
        # mu^T x >= r => -mu^T x <= -r
        G_extended = np.vstack([base_G, -mu.reshape(1,-1)])
        h_extended = np.concatenate([base_h, np.array([-r])])

        # objective => x^T Sigma x => in cvxopt form => 0.5*x^T(2Sigma)x
        Q_eff = 2.0 * Sigma
        c_eff = np.zeros(n)

        try:
            x_opt, _ = solve_qp_cvxopt(Q_eff, c_eff, A=A, b=b, 
                                       G=G_extended, h=h_extended)
            risk_val = x_opt @ Sigma @ x_opt
            ret_val = x_opt @ mu

            # accept only if ret_val >= r (numerical tolerance)
            if ret_val >= r - 1e-6:
                results_risk.append(risk_val)
                results_return.append(ret_val)
                results_weights.append(x_opt)
        except:
            # solver might fail or be infeasible
            continue

    return np.array(results_risk), np.array(results_return), np.array(results_weights)

	•	Run both WSM and ECM with different numbers of sample points (e.g., num_points=5,11,21,51).
	•	Plot the resulting Pareto fronts.
	•	Compare how many unique solutions each approach yields (some alphas can produce the same optimum).

In [ ]:
################################################################################
# CELL 7: Run & Compare For Different Calculation Limits, Plot Pareto Front
################################################################################

# Let's try a few different numbers of alpha steps for WSM
points_list = [5, 11, 21, 51]  # e.g., 5, 11, 21, 51 points

plt.figure()
for num_points in points_list:
    wsm_risk, wsm_ret, _ = wsm_portfolios(mu, Sigma, num_points=num_points)
    plt.scatter(wsm_risk, wsm_ret, label=f"WSM {num_points} pts", alpha=0.7)

# Also do the ECM approach with e.g. 11 points
ecm_risk, ecm_ret, _ = ecm_portfolios(mu, Sigma, num_points=11)
plt.scatter(ecm_risk, ecm_ret, marker='x', color='r', label="ECM 11 pts")

plt.xlabel("Portfolio Risk (Variance)")
plt.ylabel("Portfolio Return")
plt.title("Pareto Front: Weighted Sum & Epsilon-Constraint")
plt.legend()
plt.show()

################################################################################
# We can also compare how many *unique* solutions each method returns.
# E.g., for WSM with 51 alpha steps, do we get 51 unique solutions or fewer?
################################################################################

for num_points in points_list:
    wsm_risk, wsm_ret, wsm_wts = wsm_portfolios(mu, Sigma, num_points=num_points)

    # Round to mitigate floating precision issues
    risk_ret_pairs = np.round(np.column_stack((wsm_risk, wsm_ret)), 6)
    unique_pairs = np.unique(risk_ret_pairs, axis=0)
    print(f"[WSM] alpha steps = {num_points:2d} => found {len(unique_pairs)} unique (risk,return) solutions.")

ecm_risk, ecm_ret, ecm_wts = ecm_portfolios(mu, Sigma, num_points=11)
risk_ret_pairs = np.round(np.column_stack((ecm_risk, ecm_ret)), 6)
unique_ecm = np.unique

## Interpretation of the Pareto Plots and Unique Solutions

1. **Plot Description**  
   - The *x-axis* represents the **portfolio risk** (variance). Lower values mean less risk.  
   - The *y-axis* represents the **portfolio return** predicted by the model. Higher values mean higher return.  
   - Each marker represents a different **optimal portfolio** found by the optimization method (Weighted Sum or Epsilon-Constraint).

2. **Weighted Sum Method (WSM)**  
   - We vary the parameter \(\alpha \in [0,1]\) in discrete steps (e.g., 5, 11, 21, 51).  
   - For each \(\alpha\), the model tries to minimize \(\alpha \times \text{risk} - (1-\alpha)\times \text{return}\).  
   - The reported *unique* solutions are those with distinct \((\text{risk}, \text{return})\) pairs.  
   - In the figure, you see fewer points when using fewer \(\alpha\)-steps because multiple \(\alpha\) values can produce the *same* portfolio.  
   - As you increase the number of \(\alpha\)-steps, you typically discover more unique points on the Pareto front.  

3. **Epsilon-Constraint Method (ECM)**  
   - We vary a target return \(r\) and minimize \(\text{risk}\) subject to \(\text{return} \ge r\).  
   - The red “x” markers in the plot show the ECM solutions for 11 target-return steps.  
   - Notice these solutions can span a somewhat different shape or distribution than WSM because the Pareto frontier is sampled by fixing \(\text{return}\) rather than weighting risk/return directly.

4. **Observations & Conclusions**  
   - Both methods identify *efficient (Pareto-optimal) portfolios*: each point is a valid trade-off between risk and return.  
   - **WSM solutions** can “bunch up” in some regions if those risk/return objectives lead to the same optimum. Increasing the number of \(\alpha\) steps from 5 to 51 yields more unique portfolios.  
   - **ECM solutions** for 11 steps may yield a different coverage of the front.  
   - Comparing the two sets of solutions shows that the Pareto front has a characteristic shape where risk increases to achieve higher returns.  
   - Generally, you can see which **method** (and how many sampling steps) is more effective in covering the front thoroughly.  

5. **Next Steps**  
   - If you need a denser approximation, increase the number of steps (either \(\alpha\)-steps for WSM or return-target steps for ECM).  
   - If you see identical solutions repeated, that indicates multiple parameter values converge to the same optimum.  
   - You can also combine these two approaches or adopt more sophisticated sampling methods (e.g., binary search on returns, or multi-objective evolutionary algorithms) to capture the frontier more completely.